# Brain.ai Workshop with Visma

<img src="brain_ny_times_illustration.png">
Illustration source: https://www.nytimes.com/interactive/2020/12/10/us/covid-vaccine-distribution.html

### Prerequisites
* Have Python 3+ installed on your machine
* Install jupyter and check that you can open jupyter with 'jupyter notebook'
* If you're able to, work on two monitors

### Acknowlegdements

This notebook has been inspired by: https://gist.github.com/hollemawoolpert/612b51f5a489d276a4240f298a3cc9ed#file-postbuild

### Setup

In [ ]:
# Install libraries
!pip install ortools
!pip install gmaps
!pip install googlemaps
!pip install geopy
!pip install numpy
!pip install matplotlib

In [ ]:
# Enable gmaps extension
!jupyter nbextension enable --py --sys-prefix gmaps

In [ ]:
# Imports
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import gmaps
import geopy
import googlemaps
import numpy as np
import random
import math
import matplotlib.pyplot as plt

### Sections
1. Backstory
2. Preparation
3. Generate Random Baseline Solution
5. Find solution manually
4. Design and use a Genetic Algorithm
5. Solve using OR Tools

<img src="brain_ai_vaccine.jpeg">
Source: https://www.globaltimes.cn/content/1209927.shtml

## 1. Backstory

The rollout of the Covid-19 vaccine in Norway has finally started to pick up pace. A large batch of vaccines has been delivered to St. Olavs Hospital, and they need to be distributed to the vaccination sites. The sites include your favorite examination sites!

The vaccines have to be kept at 203 Kelvin, so special pupose vehicles are needed. There is a limited amount of these vehicles, and the vaccines need to be distributed swiftly.

Your have been assigned with the task of determining which route each vehicle should drive in order to minimize the time used to distribute the vaccines!

## 2. Preparation

#### Create account and get API key to use Google Distance Matrix API and Maps API

We wish to use Google's Distance Matrix API to efficiently get distances between places. We also need some other APIs. To use these APIs, you have to register your credit card information. You will get enough free credits for the purpose of this workshop, and Google promises you won't be billed without explicitly accepting that. However, if you are not comfortable with this, you can use the provided distance matrix and skip the map parts.

If you want to use the APIs, you need an account and an API key:

1. If you have never used the Google Cloud Console to create a billing account or a project, go to https://developers.google.com/maps/gmp-get-started and click "Get Started". Follow the steps.
2. Check the "Routes" and "Places" products and click "Enable".
3. Copy your API key. 
4. Try opening this in your browser:
[api-request](https://maps.googleapis.com/maps/api/distancematrix/json?origins=studentersamfundet+trondheim|gl%C3%B8shaugen+trondheim|downtown+trondheim&destinations=studentersamfundet+trondheim|gl%C3%B8shaugen+trondheim|downtown+trondheim&mode=walking&key=API_KEY), replacing `API_KEY` at the end with your API key. You should get a json distance matrix as response.
5. Assign your API key to `API_KEY` in the cell below.


In [ ]:
# TODO: Change to generic string in student version
from config import API_KEY

### Using the Maps API

Let's set some locations to visit and plot them in a map. Feel free to change locations, and run the cell below.

NB: The distance matrix API used later has a limit of 10 locations per request.

In [ ]:
# TODO: Update locations before workshop
location_names = [
    "St. Olavs Hospital",
    "Nidaros legekontor",
    "Møllenberg legesenter",
    "Sit Gløshaugen legesenter",
    "Trondheim spektrum",
    "Byåsen legesenter",
    "Valentinlyst legesenter",
    "Utleirahallen",
    "Dragvoll",
    "Sluppen"
]
depot = 0  # start and end location index

gmaps.configure(api_key=API_KEY)
geocoder = geopy.geocoders.GoogleV3(api_key=API_KEY)

locations = [geocoder.geocode(name + " Trondheim") for name in location_names]
coordinates = [(loc.latitude, loc.longitude) for loc in locations]

depot_layer = gmaps.symbol_layer(
    [coordinates[depot]],
    hover_text="Start: " + location_names[depot],
    fill_color='white',
    stroke_color='red',
    scale=8
)

stops_layer = gmaps.symbol_layer(
    coordinates[0:depot] + coordinates[depot+1:],
    hover_text=location_names[0:depot] + location_names[depot+1:],
    fill_color='white',
    stroke_color='black',
    scale=4
)

fig = gmaps.figure()
fig.add_layer(depot_layer)
fig.add_layer(stops_layer)

fig

### Using the Distance Matrix API

Next, we use the Distance Matrix api to get our distance matrix.

In [ ]:
def build_distance_matrix(coordinates, travel_mode, measure='duration'):
    """Get distance matrix for coordinates.

    Element (i,j) is the distance between the i-th and j-th coordinate.

    Args:
        coordinates: List of two-tuples of latidude/longitude-coordinates.
        measure (string): 'duration' for seconds or 'distance' for meters.

    Returns:
        Distance matrix (list of lists) of size n*n, where n==len(coordinates).
    """
    client = googlemaps.Client(key=API_KEY)
    dm_response = client.distance_matrix(origins=coordinates, destinations=coordinates, mode=travel_mode)
    dm_rows = [row['elements'] for row in dm_response['rows']]
    distance_matrix = [[item[measure]['value'] for item in dm_row] for dm_row in dm_rows]
    return distance_matrix


In [ ]:
travel_mode = 'driving'
distance_matrix = build_distance_matrix(coordinates, travel_mode)
distance_matrix

## 3. Baseline Solution
We start by creating a random solution. This will serve as a baseline; we should be able to do much better than random. A random solution will also solve as a starting point when we solve the problem using a genetic algorithm later.

In [ ]:
# Create a random solution
def create_random_solution(locations, depot, num_vehicles):
    """Create a random solution.

    Create a random solution to our vehicle routing problem.

    Args:
        locations: List of location objects.
        depot (int): Index of the depot.
        num_vehicles (int): Number of vehicles.

    Returns:
        list of ints represeting a solution (see "Chromosome Representation" below).
    """
    num_locations = len(locations)
    solution = list(range(num_locations))  # 1 depot
    for _ in range(num_vehicles - 2):
        solution.append(depot)  # num_vehicles-1 depots
    random.shuffle(solution)  # note that this is in-place
    return solution

In [ ]:
random_solution = create_random_solution(locations, depot, num_vehicles=3)
random_solution

### Evaluate the solution
Let's create some methods for helping us evaluate solutions to the VRP.

In [ ]:
def get_routes_dict(solution, depot):
    """Get a dictionary representing the solution.

    Get a representation of the routes that is simpler to read and work with.

    Args:
        solution (list of ints): A solution chromosome.
        depot (int): Index of the depot.

    Returns:
        A dictionary with one element of ordered indices for each vehicle route.
    """
    routes = dict()
    route_num = 0
    route = [depot]
    for i in solution:
        route.append(i)
        if i == depot:
            routes[route_num] = route
            route_num += 1
            route = [depot]
    route.append(depot)
    routes[route_num] = route
    return routes


In [ ]:
random_routes = get_routes_dict(random_solution, depot)
random_routes

In [ ]:
def get_route_distance(route, distance_matrix):
    """Get the distance of a route.

    Args:
        route (list of ints): A vehicle route.
        distance_matrix (list of lists of ints): A distance matrix.

    Returns:
        int: Distance of the route.
    """
    distance = 0
    for k in range(len(route) - 1):
        i_from = route[k]
        i_to = route[k + 1]
        distance += distance_matrix[i_from][i_to]
    return distance


def get_max_route_distance(routes, distance_matrix):
    """Get the longest route distance.

    Get the distance of the longest route in a set of routes.

    Args:
        routes (dict): Vehicle routes.
        distance_matrix (list of lists of ints): A distance matrix.

    Returns:
        int: Distance of the longest route.
    """
    return max([get_route_distance(route, distance_matrix) for route in routes.values()])


def pretty_print(routes, location_names, distance_matrix):
    """Create a nice printout of a set of routes.

    Args:
        routes (dict): Vehicle routes.
        location_names (list of str): Names of the locations.
        distance_matrix (list of lists of ints): A distance matrix.
    """
    for vehicle, route in routes.items():
        route_distance = get_route_distance(route, distance_matrix)
        if route_distance == 0:
            print(f"Vehicle {vehicle} not used\n")
            continue
        route_str = str(location_names[route[0]])
        for i in route[1:]:
            route_str += " -> " + location_names[i]
        print(
            f"Route for vehicle {vehicle}:\n" +
            route_str + "\n" +
            f"Distance of route: {route_distance}\n"
        )
    print(f"Maximum of the route distances: {get_max_route_distance(routes, distance_matrix)}")


In [ ]:
pretty_print(random_routes, location_names, distance_matrix)

In [ ]:
def map_solution(routes, coordinates, depot):
    """Create a map figure showing vehicle routes.

    Args:
        routes (dict): Vehicle routes.
        location_names (list of str): Names of the locations.append
        distance_matrix (list of lists of ints): A distance matrix.

    Returns:
        gmaps figure object.
    """
    fig = gmaps.figure()
    colors = ['blue','red','green','#800080','#000080','#008080']
    assert len(colors) >= len(routes), "More colors needed"
    for vehicle_id in routes:
        waypoints = []
        depot_coordinates = coordinates[depot]
        
        # skip depot (occupies first and last index)
        for coordinate_index in routes[vehicle_id][1:-1]:
            waypoints.append(coordinates[coordinate_index])
        
        if len(waypoints) == 0:
            print('Empty route:', vehicle_id)
        else:
            route_layer = gmaps.directions_layer(
                depot_coordinates, waypoints[-1], waypoints=waypoints[0:-1], show_markers=True,
                stroke_color=colors[vehicle_id], stroke_weight=5, stroke_opacity=0.5,
                travel_mode=travel_mode.upper())
            fig.add_layer(route_layer)
            
            # complete the route from last location to depot
            return_layer = gmaps.directions_layer(
                waypoints[-1], depot_coordinates, show_markers=False,
                stroke_color=colors[vehicle_id], stroke_weight=5, stroke_opacity=0.5,
                travel_mode=travel_mode.upper())
            fig.add_layer(return_layer)
    
    return fig


In [ ]:
map_solution(random_routes, coordinates, depot)

Does this look like a good solution? In that case, try again...

Combinatorial (or discrete) optimization problems, such as this one, have large solution spaces even for small inputs. So a random solution will most likely not be a good one.

BONUS TASK: Quantify the complexity of the VRP.

Answer: Let n be the number of stops, excluding the depot, and v the number of vehicles. v^n possible assignements of stops to vehicles. Assume longest route has ceil(n/v) stops, with ceil(n/v)! possible orders. Then we get O( v^n \* ceil(n/v)! ). Alternatively: O( v^n \* (round(n/v)!)^v ). Or worst case upper bound (not tight): O( v^n * n! )

## 4. Manually solve the problem
Let's try to solve the problem manually by inspecting the map and decide which routes the vehicles should make
#TODO Sjekk at man ikke kan redigere hovedfila, men at man får lov til å kopiere den. 

TASK: 

1. Click the following link to open up a Google Drawing of the VRP instance. https://docs.google.com/drawings/d/1L0bQ9KUoX3CBnvLtdrDxZ2nC4Wq50ZfdqhvpKKT10mc/edit?usp=sharing
2. Click 'File' in the top left corner and then 'Make a copy'. 
3. Inspect the problem and try to draw the best route for each of the three vehicles (Select the dropdown menu at 'Line' and select 'Scribble' to draw freely. 
4. Insert your solution into manual_solution below. This represents the routes taken by each vechicle. 
5. Evaluate your solution.

In [ ]:
# Insert your solution here. 
manual_routes = {0: [0, 4, 5, 0], 1: [0, 9, 7, 8, 3, 0], 2: [0, 1, 2, 6, 0]}

In [ ]:
pretty_print(manual_routes, location_names, distance_matrix)

In [ ]:
map_solution(manual_routes, coordinates, depot)

## 5. Solving with a genetic algorithm

### Chromosome Representation

A solution to this problem can be represented in a one-dimensional matrix of size (1, n) where n = (k-1) + c, where k is the number of vehicles and c is the number of destinations. 

A solution is represented as a permutation of integers, where each integer represents a location (destination or depot). The depot values is used to delimit different routes. 

Example: 

[2,1,0,4,3,0,5] 

Vehicle 1: 0-2-1-0
Vehicle 2: 0-4-3-0
Vehicle 3: 0-5-0

Each solution will have to end with a zero (go back to depot), so we can omit this from the solution representation. 

In [ ]:
class Chromosome:
    """The class that contains a chromosome, i.e. a solution to the problem
    
    Attributes:
        route_representation (list): List representing the route of the vehicles
        route_dict (dict): A dictionary with one element of ordered indices for each vehicle route.
        longest_route_distance (float): Longest route a vehicle must make in a solution
        fitness (float): Fitness value of the chromosome
    """
    
    def __init__(self, route_representation, route_dict):
        """Initializes a chromosome.
        """
        self.route_representation = route_representation
        self.route_dict = route_dict
        self.longest_route_distance = None
        self.fitness = None
        
    def __str__(self):
        """For outputting a string of the Chromosome attributes when printing.
        """
        output_string = str(self.route_representation) + '\n' + str(self.route_dict) + '\n'
        output_string += 'fitness: ' + str(self.fitness) + '\nlongest_route_distance: ' + stl(self.longest_route_distance)
        return output_string
        

### Algorithm
Implement the GA algorithm. Study the flow chart below and the comments in the code and make sure you understand how the algorithm works.

<img src="ga.png">

In [ ]:
def ga_solver(
    locations,
    depot,
    num_vehicles,
    distance_matrix,
    population_size=50,
    num_generations=50,
    elite_size=2,
    tournament_k=3,
    tournament_p=0.95,
    mutation_rate=0.8,
    mutation_swaps=1,
):
    """Main GA algorithm, performing all the steps of the genetic algorithm
    
    Args: 
        locations: List of location objects.
        depot (int): Index of the depot.
        num_vehicles (int): Number of vehicles.
        distance_matrix (list of lists of ints): A distance matrix.
        population_size (int): Size of the population.
        num_generations (int): Number of generations to evolve.
        elite_size (int): Number of best solutions to bring directly into next generation.
        tournament_k (int): Number of competitors in the tournament selection.
        tournament_p (float): Probability of the best competitor winning the tournament.
        mutation_rate (float): Probability of performing mutation on a Chromosome.
        mutation_swaps (int): Number of swaps to do in swap mutation.
        
    Returns:
        dict: The chromosome with the best solution.
    """
    # Create initial random population
    population = initialize_population(locations, depot, num_vehicles, population_size)
    
    # Calculate fitness of population
    for chromosome in population:
        calculate_fitness(chromosome, distance_matrix)

    generation_bests = {'fitness': [], 'max_distance': []}

    # Loop for evolving the populations through generations
    for generation in range(num_generations):
        
        # Generate new generation
        population = evolve_population(
            population, elite_size, tournament_k, tournament_p, mutation_rate, mutation_swaps, depot
        )
        
        # calculate fitness
        for chromosome in population:
            calculate_fitness(chromosome, distance_matrix)
        
        # store best values
        max_distances = np.array([chromosome.longest_route_distance for chromosome in population])
        best_i = max_distances.argmin()
        chromosome = population[best_i]
        generation_bests['fitness'].append(chromosome.fitness)
        generation_bests['max_distance'].append(chromosome.longest_route_distance)

    return population[best_i], generation_bests


### Initialize population
Random initial population - Use the same approach as in generating a random solution.

TASK: Implement the content of the for loop in the method below. Hint: Use `create_random_solution()` and `get_routes_dict()`. You might want to test that the method works as expected.

In [ ]:
def initialize_population(locations, depot, num_vehicles, population_size):
    """Initialize population of Chromosome instances.
    
    Args:
        locations (list): Location objects.
        depot (int): Index of the depot.
        num_vehicles (int): Number of vehicles.
        population_size (int): Size of the population.
        
    Returns
        List of Chromosome instances.
    """
    population = []
    for i in range(population_size):

        ## START CODE HERE ##

        # Create a random solution
        route_representation = create_random_solution(locations, depot, num_vehicles)

        # Convert to dictionary representation
        route_dict = get_routes_dict(route_representation, depot)

        # Append intital random solution to population
        population.append(Chromosome(route_representation, route_dict))

        ## END CODE HERE ##

    assert len(population) == population_size   
    return population

### Evolve population

Evolving the population involves some steps we have not implemented yet: getting elites, (tournament) selection and mutating the new generation. We will get to these methods below. You can input the missing code immediately by just looking at what parameters are needed for the mentioned methods, or you can skip the implementation task here for now and come back to it later.

For now, the most important thing is that you read through `evolve_population()` to get an understanding of the main algorithm.

TASK: Implement the indicated steps in `evolve_population()` below. Hint: Use `get_elites()`, `tournament_selection()` and `mutate_population()`.

In [ ]:
def evolve_population(
    population, elite_size, tournament_k, tournament_p, mutation_rate, mutation_swaps, depot
): 
    """Take in the population and perform all the steps to return a new generation. 
    
    1. Add elites
    2. Select parents
    3. Crossover
    4. Mutation
    5. Return new generation
    
    Args: 
        population (list): List of Chromosome instances
        depot (int): Index of the depot.
        elite_size (int): Number of best solutions to bring directly into next generation.
        tournament_k (int): Number of competitors in the tournament selection.
        tournament_p (float): Probability of the best competitor winning the tournament.
        mutation_rate (float): Probability of performing mutation on a Chromosome.
        mutation_swaps (int): Number of swaps to do in swap mutation.
        
    Returns: 
        List of Chromosome instances of the next iteration.
    """
    next_generation = []
    
    # Add elites
    if elite_size > 0:

        ## START CODE HERE
        elites = get_elites(population, elite_size)
        next_generation = elites
        ## END CODE HERE

    # Parent (tournament) selection
    mating_pool_size = len(population) - elite_size

    ## START CODE HERE
    mating_pool = tournament_selection(population, tournament_k, tournament_p, mating_pool_size)
    ## END CODE HERE
    
    # Generate children
    # NOTE: Children are just a list of the route_representations, not Chromosome instances
    children = breed_parents(mating_pool, depot)
    
    # perform mutation
    
    ## START CODE HERE
    mutated_route_representations = mutate_population(children, mutation_rate, mutation_swaps)
    ## END CODE HERE
    
    # transform the mutated route representations into Chromosome instances
    for route_representation in mutated_route_representations:
        route_dict = get_routes_dict(route_representation, depot)
        next_generation.append(Chromosome(route_representation, route_dict))

    assert len(next_generation) == len(population)
    return next_generation

### Fitness evaluation
Find the longest route in the solution. 

Here, if we get imbalanced routes, we can add a overcapacity penalty for the number of nurses that has to travel to more than roof(c/k) destinations. 

In [ ]:
def calculate_fitness(chromosome, distance_matrix):
    """Calculate the fitness and assign it to the fitness attribute of the chromosome.
    
    Args:
        chromosome (Chromosome): The Chromosome instance.
        distance_matrix (list of lists of ints): A distance matrix.
    """
    chromosome.longest_route_distance = get_max_route_distance(chromosome.route_dict, distance_matrix)
    chromosome.fitness = 1/chromosome.longest_route_distance

### Elites

In [ ]:
def get_elites(population, elite_size): 
    """Get the best chromosomes from the population.

    Args:
        population (list): List of the Chromosome instances.
        elite_size (int): Number of best solutions to bring directly into next generation.
        
    Returns: 
        List of the elite Chromosomes.
    """
    # sort population descendingly in-place according to fitness value
    population.sort(key=lambda x: x.fitness, reverse=True)
    elites = population[:elite_size]
    return elites

### Selection
Select mating pool using tournament selection. See the illustration below and the comments in the code for an explanation.

TASK: Finish the tournament selection method below. Hint: `random.sample()` and `np.random.choice()` are methods that might come in handy.

<img src="tournament_selection.png">

In [ ]:
def tournament_selection(population, tournament_k, tournament_p, mating_pool_size): 
    """Takes in the entire population and returns a mating pool of the same size. 
    
    The mating pool is the parents selected for breeding.
    
    Args:
        population (list): List of the Chromosome instances.
        tournament_k (int): Number of competitors in the tournament selection.
        tournament_p (float): Probability of the best competitor winning the tournament.
        mating_pool_size: (int): The size of the mating pool, i.e. the number of parents selected for breeding.
    
    Returns: 
        List of Chromosome instances that form the mating pool.
    """
    mating_pool = []
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    # While the mating pool is not full
    while len(mating_pool) < mating_pool_size:
        
        ## START CODE HERE ##
        # Select competitors at random
        competitors = random.sample(population, tournament_k)
        
        # Sort competitors according to fitness
        competitors.sort(key=lambda x: x.fitness, reverse=True)
        
        # Get the probabilities for the competitors. probability: p * (1-p)**i
        probabilities = np.empty(tournament_k)
        for i in range(tournament_k):
            probabilities[i] = tournament_p * (1-tournament_p)**i
        
        # Normalize the probabilities
        probabilities = probabilities / probabilities.sum()
        
        # Choose the winner according to the propabilities
        winner = np.random.choice(competitors, p=probabilities)
        ## END CODE HERE ##
        
        mating_pool.append(winner)
    
    assert len(mating_pool) == mating_pool_size
    return mating_pool

### Crossover
Use ordered crossover. Our chromosome representation makes this relatively straight-forward, but we still need to watch our step when implementing it. We have done this part for you.

BONUS TASK: Try using egde combination crossover.

<img src="ordered_crossover.png">

From: https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35

In [ ]:
def breed_parents(mating_pool, depot):
    """Performs the crossover and returns the list of children chromosomes.
    
    Args:
        mating_pool (list): List of Chromosome instances.
        depot (int): Index of the depot.
    
    Returns:
        List of Chromosome instances of the children produced by crossover.
    """
    children = []
    
    # shuffle the order of the parents in-place
    random.shuffle(mating_pool)
    
    # each crossover produces two children, so the crossover should be executed (size/2) times
    num_iterations = math.ceil(len(mating_pool)/2)
    for i in range(num_iterations):
        parent1 = mating_pool[i]
        parent2 = mating_pool[len(mating_pool)-i-1]
        child1, child2 = ordered_crossover(parent1, parent2, depot)
        children.append(child1)
        children.append(child2)

    # If mating pool is of odd length, we have one excess child. So we leave it behind...
    if (len(children) > len(mating_pool)):
        children.pop()
    
    return children

In [ ]:
def ordered_crossover(parent1_chromosome, parent2_chromosome, depot):
    """Performs ordered crossover and returns the route_representations of the two children.
    
    Args:
        parent1_chromosome (Chromosome): Parent 1 Chromosome.
        parent2_chromosome (Chromosome): Parent 2 Chromosome.
        depot (int): Index of the depot.
        
    Returns: 
        Two lists with route_representations of the children (not Chromosome instances).
    """
    # copy the route representations from the parents
    parent1 = parent1_chromosome.route_representation.copy()
    parent2 = parent2_chromosome.route_representation.copy()
    
    # Replace the depot values with strings to make them unique (ordered crossover requires unique cells)
    def replace_depot_values(route):
        counter = 1
        for idx, value in enumerate(route):
            if value == depot:
                route[idx] = 'd' + str(counter)
                counter = counter + 1
    replace_depot_values(parent1)
    replace_depot_values(parent2)

    # generate random index slicing points
    length = len(parent1)
    start_slice = random.randint(0, length - 1)
    end_slice = random.randint(start_slice + 1, length)
    
    # get the values from start (included) to stop (excluded)
    child1_middle = parent1[start_slice:end_slice]
    child2_middle = parent2[start_slice:end_slice]
    
    # get the values to fill in from the other parent in each child
    child1_fill = [i for i in parent2 if i not in child1_middle]
    child2_fill = [i for i in parent1 if i not in child2_middle]
    
    # put together the fill values and the unchanged part between the slice points
    child1 = child1_fill[:start_slice] + child1_middle + child1_fill[start_slice:]
    child2 = child2_fill[:start_slice] + child2_middle + child2_fill[start_slice:]
    
    # transform the depot values back to zeros
    child1 = [0 if str(value).startswith('d') else value for value in child1]
    child2 = [0 if str(value).startswith('d') else value for value in child2]
    
    return child1, child2
    

### Mutation
Swap two places (ore more) with each other in one individual.

TASK: Finish `swap_mutate()` below.

<img src="swap_mutation.jpg">

From: https://www.theprojectspot.com/tutorial-post/applying-a-genetic-algorithm-to-the-travelling-salesman-problem/5

In [ ]:
def mutate_population(children, mutation_rate, mutation_swaps):
    """Mutate population.

    Takes in children which is a list of route_representations of the children
    (because we don't bother with computing fitness before we have done mutation).
    
    Args: 
        children (list): List of lists containing the route representations of the children.
        mutation_rate (float): Probability of performing mutation on a Chromosome.
        mutation_swaps (int): Number of swaps to do in swap mutation.
    
    Returns:
        List of lists with the route representations for the next generation.
    """
    mutated_population = []
    
    for child in children:
        if (random.random() < mutation_rate):
            mutated_child = swap_mutate(child, mutation_swaps)
            mutated_population.append(mutated_child)
        else:
            mutated_population.append(child)
    
    return mutated_population

In [ ]:
def swap_mutate(route_representation, number_of_swaps):
    """Run swap mutation on a route_representation list.
    
    Args:
        route_representation (list): List of the route representation for a solution.
        number_of_swaps (int): Number of swaps to do in swap mutation.
    
    Returns: 
        List containing the route representation of the mutated child.
    """
    for _ in range(number_of_swaps): 
        
        ## START CODE HERE ##
        # choose two random indexes to swap
        index1_to_swap = random.choice(list(range(len(route_representation))))
        index2_to_swap = random.choice(list(range(len(route_representation))))
        
        # temporarily store the values in these indices
        destination1 = route_representation[index1_to_swap]
        destination2 = route_representation[index2_to_swap]
        
        # switch the values in the two indices
        route_representation[index1_to_swap] = destination2
        route_representation[index2_to_swap] = destination1
        ## END CODE HERE ##
        
    return route_representation

### Solve using the genetic algorithm

In [ ]:
ga_solution, history = ga_solver(locations, depot, num_vehicles=3, distance_matrix=distance_matrix)

In [ ]:
# Plot how the solution improves through the generations
plt.figure()
plt.plot(history['fitness'])
plt.xlabel('generation')
plt.ylabel('fitness')
plt.figure()
plt.plot(history['max_distance'], )
plt.xlabel('generation')
plt.ylabel('max distance')

In [ ]:
ga_routes = ga_solution.route_dict
pretty_print(ga_routes, location_names, distance_matrix)
map_solution(ga_routes, coordinates, depot)

How does this solution compare to your manual solution? Is there anything you could do to increase the likelihood of finding a good solution - and at what cost?

## 6. Solving with OR Tools

It this section we will solve the same problem as before, but with the "plug-and-play" Google OR-tools python package. 

Comparing the amount of code and work we need when using OR-tools, compared to when designing an algorithm ourselves, illustrates the importance of not reinventing the wheel. More often than not the problems you are trying to solve are solved by someone else before you. This has to be taken into account before putting in a huge effort to build something from scratch.

### Model

First, we build the OR-tools model. See the [OR-tools documentation](https://developers.google.com/optimization/routing/vrp) for a more detailed explanations of the steps.

In [ ]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return distance_matrix[from_node][to_node]


In [ ]:
num_vehicles = 3

# Create the routing index manager
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, depot)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Register the distance callback. The solver will use this to get distances
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc - which is the distance
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add duration contraint as a dimension
dimension_name = 'Duration'
max_distance = max([max(distance_row) for distance_row in distance_matrix])
max_vehicle_travel_distance = max_distance * 2 * len(distance_matrix) // num_vehicles
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    max_vehicle_travel_distance,
    True,  # start cumul to zero
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

### Solve using OR-tools

We find a solution using the model defined above.

In [ ]:
# Set a baseline solution with heuristic
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [ ]:
# Solve the problem, i.e. minimize duration of longest route
or_solution_object = routing.SolveWithParameters(search_parameters)

### Evaluate the solution

TASK: Fill in the missing code for printing and mapping the solution below.

In [ ]:
def extract_routes(num_vehicles, manager, routing, solution):
    """Extract routes dictionary from solution object.
    """
    routes = {}
    for vehicle_id in range(num_vehicles):
        routes[vehicle_id] = []
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            routes[vehicle_id].append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
        routes[vehicle_id].append(manager.IndexToNode(index))
    return routes


In [ ]:
or_routes = extract_routes(num_vehicles, manager, routing, or_solution_object)

## INSERT CODE HERE ##
pretty_print(or_routes, location_names, distance_matrix)
map_solution(or_routes, coordinates, depot)
## END CODE HERE ##

Some obligatory reflection:
- How does this solution compare to you GA solution?
- Are any of them optimal?
- Why does/doesn't OR-tools provide on optimal solution?

### Compare solutions

In [ ]:
random_max_dist = get_max_route_distance(random_routes, distance_matrix)
manual_max_dist = get_max_route_distance(manual_routes, distance_matrix)
ga_max_dist = get_max_route_distance(ga_routes, distance_matrix)
or_max_dist = get_max_route_distance(or_routes, distance_matrix)
results = [random_max_dist, manual_max_dist, ga_max_dist, or_max_dist]

ind = range(4)
_ = plt.bar(ind, results)
_ = plt.ylabel("Longest route distance")
_ = plt.xticks(ind, ['Random', 'Manual', 'GA', 'OR-tools'])

How do the solutions compare? Which approach is the most scalable? Below we have outlined some extra constraints/dimensions complicating things further. Which approach would you go for if you were to take these into account?

## 7. Extra challenges

Here are some ideas for extra challenges to have a go at if you finished early, or if you want to play some more around with different parts of what we have gone through. They should also illustrate well that there is almost no limit for how complex route optimization can become.

- Host this notebook online with [binder](https://mybinder.org/)
- Add stop/offload times. Hint: Start with the distance matrix.
- Add more locations. Hint: The free distance matrix API has a limit for how many distances you can get in a single request, but we are free to post multiple requests :)
- Add capacity constraint: Each location needs a number of vaccines, and each vehicle has a limited capacity. Use OR-tools and/or add support to the genetic algorithm. 
- Maybe the vehicles can refill when they are empty?
- Time windows: Some locations can only receive the vaccines at certain times.
- Handling the vaccines is complex, so at some locations two vehicles must be there together for the entire stop time before they can move on.